In [1]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.4 MB/s eta 0:00:00


In [2]:
#Dependancies
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import keras_tuner as kt  # Hyperparameter tuning
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

print("Path to dataset files:", path)

100%|██████████| 750M/750M [00:09<00:00, 81.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jessicali9530/stanford-dogs-dataset/versions/2


In [4]:
#Split Data into training and testing groups

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_PATH = path

# Apply rescaling (since your images are already augmented)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% validation data
)

# Create Train Dataset
train_data = datagen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='training'
)

# Create Validation Dataset
val_data = datagen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='validation'
)


Found 16464 images belonging to 2 classes.
Found 4116 images belonging to 2 classes.


In [5]:
#Not sure if this is needed
IMG_SIZE = (224, 224)  # Image size
INPUT_SHAPE = (224, 224, 3)  # Image dimensions (RGB)
NUM_CLASSES = 2  # Replace with the number of dog breeds in your dataset


In [6]:
def create_model(hp):
    model = Sequential()

    # First Conv Layer with tunable filters & activation
    model.add(Conv2D(
        filters=hp.Int('conv1_filters', min_value=16, max_value=128, step=32),
        kernel_size=(3,3), activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid']),
        input_shape=INPUT_SHAPE
    ))
    model.add(MaxPooling2D(2,2))

    # Allow kerastuner to decide number of Conv layers
    for i in range(hp.Int('num_conv_layers', 1, 4)):
        model.add(Conv2D(
            filters=hp.Int(f'conv_filters_{i}', min_value=16, max_value=128, step=32),
            kernel_size=(3,3), activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
        ))
        model.add(MaxPooling2D(2,2))

    model.add(Flatten())

    # Dense layers (fully connected)
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i}', min_value=64, max_value=512, step=64),
            activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
        ))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))

    # Output layer for multi-class classification
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [7]:
# def create_model(hp):
#     model = Sequential()

#     # First Conv Layer with optimized filters & fixed activation (ReLU is best)
#     model.add(Conv2D(
#         filters=hp.Int('conv1_filters', min_value=32, max_value=64, step=32),
#         kernel_size=(3,3), activation='relu',
#         input_shape=INPUT_SHAPE
#     ))
#     model.add(MaxPooling2D(2,2))

#     # Reduce number of Conv layers (max 2)
#     for i in range(hp.Int('num_conv_layers', 1, 2)):
#         model.add(Conv2D(
#             filters=hp.Int(f'conv_filters_{i}', min_value=32, max_value=64, step=32),
#             kernel_size=(3,3), activation='relu'
#         ))
#         model.add(MaxPooling2D(2,2))

#     model.add(Flatten())

#     # Reduce Dense layers & neuron count
#     for i in range(hp.Int('num_dense_layers', 1, 2)):
#         model.add(Dense(
#             units=hp.Int(f'dense_units_{i}', min_value=32, max_value=256, step=64),
#             activation='relu'
#         ))
#         model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.3, max_value=0.5, step=0.1)))

#     # Output layer for multi-class classification
#     model.add(Dense(NUM_CLASSES, activation='softmax'))

#     # Compile the model
#     model.compile(
#         optimizer='adam',
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )

#     return model


In [8]:
from kerastuner.tuners import RandomSearch

<ipython-input-8-94471a811b41>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [9]:
# Define tuner with faster settings
tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=10,  # ⬇️ Less models to test (faster tuning)
    executions_per_trial=1,  # ⬇️ Train each model ONCE instead of twice!
    directory='kerastuner_results',
    project_name='dog_breed_classifier'
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Run tuning
tuner.search(train_data, validation_data=val_data, epochs=5)

Trial 10 Complete [00h 04m 10s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 49m 28s


In [11]:
# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

In [12]:
# Train the best model
history = best_model.fit(train_data, validation_data=val_data, epochs=20)

Epoch 1/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 57s 100ms/step - accuracy: 0.9856 - loss: 0.0458 - val_accuracy: 1.0000 - val_loss: 3.5864e-04
Epoch 2/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 50s 98ms/step - accuracy: 1.0000 - loss: 4.5226e-04 - val_accuracy: 1.0000 - val_loss: 1.1980e-04
Epoch 3/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 47s 91ms/step - accuracy: 1.0000 - loss: 1.8036e-04 - val_accuracy: 1.0000 - val_loss: 6.0199e-05
Epoch 4/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 48s 93ms/step - accuracy: 1.0000 - loss: 9.8846e-05 - val_accuracy: 1.0000 - val_loss: 3.5881e-05
Epoch 5/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 48s 94ms/step - accuracy: 1.0000 - loss: 6.0373e-05 - val_accuracy: 1.0000 - val_loss: 2.3126e-05
Epoch 6/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 48s 93ms/step - accuracy: 1.0000 - loss: 4.1605e-05 - val_accuracy: 1.0000 - val_loss: 1.5616e-05
Epoch 7/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 50s 97ms/step - accuracy: 1.0000 - loss: 3.0327e-05 - val_accuracy: 1.0000 - val_loss: 1.0848e-05
Epoch 8/20
515/515 ━━━━━━━━━━━━━━━━━━

In [13]:
# Save the best model
best_model.save("best_dog_breed_classifier.h5")

In [14]:
from google.colab import files


# Download the saved model to your local machine
files.download("best_dog_breed_classifier.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>